In [ ]:
import torch
from PIL import Image

# Model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

print(model.classes)

# Images
imgs = Image.open('faces/22.jpg')

# Inference
results = model([imgs])

# Results
results.print()
results.show()  # or .show()

results.xyxy[0]  # img1 predictions (tensor)
results.pandas().xyxy[0]  # img1 predictions (pandas)

In [ ]:
import pandas as pd

print(results.pandas().xyxy[0])
df = pd.DataFrame(results.pandas().xyxy[0])
df.drop('name', axis=1, inplace=True)
mask = df['class'] != 0
df.drop(df[mask].index, inplace=True)
print(df)

In [ ]:
# %cd .
# !git clone https://github.com/ifzhang/ByteTrack.git
# %cd workspace/ByteTrack

# # workaround related to https://github.com/roboflow/notebooks/issues/80
# !sed -i 's/onnx==1.8.1/onnx==1.9.0/g' requirements.txt

# !pip3 install -q -r requirements.txt
# !python3 setup.py -q develop
# !pip install -q cython_bbox
# !pip install -q onemetric
# # workaround related to https://github.com/roboflow/notebooks/issues/112 and https://github.com/roboflow/notebooks/issues/106
# !pip install -q loguru lap thop

from IPython import display
display.clear_output()


import sys
sys.path.append(f"/workspace/ByteTrack")


import yolox
print("yolox.__version__:", yolox.__version__)

In [ ]:
from yolox.tracker.byte_tracker import BYTETracker, STrack
from onemetric.cv.utils.iou import box_iou_batch
from dataclasses import dataclass


@dataclass(frozen=True)
class BYTETrackerArgs:
    track_thresh: float = 0.25
    track_buffer: int = 30
    match_thresh: float = 0.8
    aspect_ratio_thresh: float = 3.0
    min_box_area: float = 1.0
    mot20: bool = False

In [ ]:
from pathlib import Path
import cv2
import torch
from yolox.tracker.byte_tracker import BYTETracker
import pandas as pd

# Parameters

# Tracker
# tracker = BYTETracker(None, fps)

# Model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

# Путь к видеофайлу
video_path = 'mp4/Video_example.mp4'  # Замените путь на ваш видеофайл

# Загрузка видео
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

# Создание объекта VideoWriter для записи результата
output_path = 'output.mp4'  # Замените путь по своему усмотрению
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))
 
while cap.isOpened():
    ret, frame = cap.read()
    
    if not ret:
        break
    
    # Детекция объектов
    results = model([frame])
    
    # Список bboxes, принадлежащих классу 0 ('person')
    df = pd.DataFrame(results.pandas().xyxy[0])
    df.drop('name', axis=1, inplace=True)
    mask = df['class'] != 0
    df.drop(df[mask].index, inplace=True)
    
#     for xmin, ymin, xmax, ymax, confidence, _ in df:
#         online_targets = tracker.update(dets, info_imgs, img_size)
        
    # Отрисовка bounding boxes на изображении
    frame_with_boxes = results.render()[0].astype('uint8')
    
    # Запись кадра с bounding boxes в выходной файл
    output.write(frame_with_boxes)

    
# Освобождение ресурсов
cap.release()
output.release()
cv2.destroyAllWindows()

In [ ]:
import sys
sys.path.append("/workspace/yoloface/")
from yoloface.face_detector import YoloDetector
import numpy as np
from PIL import Image
import cv2

model = YoloDetector(target_size=720, device="cuda:0", min_face=5)
orgimg = np.array(Image.open('faces/22.jpg'))
bboxes,points = model.predict(orgimg)

# Рисование bounding boxes
image = orgimg.copy()
for bbox in bboxes[0]:
    xmin, ymin, xmax, ymax = bbox
    cv2.rectangle(image, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)

print(bboxes[0])

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.imshow(image)
plt.show()

In [ ]:
import lap
import sys
sys.path.append("/workspace/yoloface/")
import cv2
import torch
from yolox.tracker.byte_tracker import BYTETracker
import pandas as pd
from yoloface.face_detector import YoloDetector
from PIL import Image
import numpy as np

# Parameters
class ByteTrackArgument:
    track_thresh = 0.5 # High_threshold
    track_buffer = 50 # Number of frame lost tracklets are kept
    match_thresh = 0.8 # Matching threshold for first stage linear assignment
    aspect_ratio_thresh = 10.0 # Minimum bounding box aspect ratio
    min_box_area = 1.0 # Minimum bounding box area
    mot20 = False # If used, bounding boxes are not clipped.

MIN_THRESHOLD = 0.001

# Model
model = YoloDetector(target_size=720, device="cuda:0", min_face=80)

# Путь к видеофайлу
video_path = 'mp4/Video_example.mp4'  # Замените путь на ваш видеофайл

# Загрузка видео
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

# Tracker
# tracker = BYTETracker(ByteTrackArgument)

# Создание объекта VideoWriter для записи результата
output_path = 'output.mp4'  # Замените путь по своему усмотрению
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))
 
while cap.isOpened():
    ret, frame = cap.read()
    
    if not ret:
        break
    # Load the image and convert it to a PIL image
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#     frame = Image.fromarray(frame)
    # Детекция объектов
    bboxes, points = model.predict(frame)
    
    # Рисование bounding boxes
    frame_with_boxes = frame.copy()
    for bbox in bboxes[0]:
        xmin, ymin, xmax, ymax = bbox
        cv2.rectangle(frame_with_boxes, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
    
#     for xmin, ymin, xmax, ymax in bboxes[0]:
#         online_targets = tracker.update(np.array([[xmin, ymin, xmax, ymax, 1.0]]), [frame_height, frame_width], [frame_height, frame_width])
#         print(online_targets)
    # Запись кадра с bounding boxes в выходной файл
    output.write(frame_with_boxes)

# Освобождение ресурсов
cap.release()
output.release()
cv2.destroyAllWindows()

In [ ]:
from IPython.display import Video

Video("output.mp4", embed=True)

In [ ]:
# == Display results ==
from IPython.display import HTML
from base64 import b64encode

video_path = "output.mp4"

mp4 = open(video_path, "rb").read()
data_url = "data:video/webm;base64," + b64encode(mp4).decode()
HTML(f"""
<video width=600 controls>
      <source src="{data_url}" type="video/webm">
</video>
""")

In [ ]:
from tracker.byte_tracker import BYTETracker

images = 
tracker = BYTETracker(args)
for image in images:
   dets = model(image)
   online_targets = tracker.update(dets, info_imgs, img_size)